# Classification performance of the keyword search algorithm

The data and code here produces the in-sample and out-of-sample classification performance statistics.

I use keyword search to find forward-looking sentences. The keyword list for forward-looking is uploaded [here](https://github.com/yiyangw2/time_frame_gold_corpus/blob/main/fls_terms2.txt). Besides, I determine that if a sentence contains words of years for the coming ten years, it is also forward-looking. The word list is adapted from [Muslu et al. [2015]](https://doi.org/10.1287/mnsc.2014.1921).

Among the forward-looking statements, I use a time frame keyword list to identify those with time frames.
The keyword list for time frames is uploaded [here](https://github.com/yiyangw2/time_frame_gold_corpus/blob/main/qt_terms2.txt).
The time frame keyword list is adapted from [Brochet, Loumioti and Serafeim [2015]](https://doi.org/10.1007/s11142-015-9329-8).

In [1]:
import pandas as pd
gold_standard = pd.read_excel("Gold Standard Corpus.xlsx")
gold_standard = gold_standard[['file_name', 'section', 'context', 
                               'speaker_number', 'future_year',
                               'speaker_text', 'fl', 'qt', 'fl_qt', 'training sample']]
gold_standard

,file_name,section,context,speaker_number,future_year,speaker_text,fl,qt,fl_qt,training sample
0,12614899_T,1,qa,23,2019,Okay.,0,0,0,0
1,12614899_T,1,qa,23,2019,"And lastly from me, can you kind of give us a ...",0,0,0,1
2,12614899_T,1,qa,23,2019,What's the share of wallet for Knoll products?,0,0,0,1
3,702568_T,1,qa,109,2003,Okay.,0,0,0,1
4,702568_T,1,qa,109,2003,So I don't think anybody said much more.,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
1283,2723598_T,1,pres,5,2010,"With that, let's open the call to your questions.",0,0,0,0
1284,2723598_T,1,pres,5,2010,"Yvette, please review the Q&A procedure.",0,0,0,1
1285,739440_T,1,qa,154,2003,This is a follow up question from Chris Joseph...,0,0,0,1
1286,739440_T,1,qa,154,2003,from [inaudible].,0,0,0,1


The underlying text associated with the gold standard is found in [`Gold Standard Corpus.xlsx`](https://github.com/yiyangw2/time_frame_gold_corpus/blob/main/Gold%20Standard%20Corpus.xlsx). It is composed of 200 randomly picked up paragraphs of the management (1288 sentences). As you can see, each sentence has identifiers for where it comes from. And values for `fl_cal`,`qt_cal` and `fl_qt_cal`, reflecting whether the algorithm determines that whether the sentence is forward-looking, whether it contains time frames and whether it is a forward-looking statement with time frames. Similarly, each observation has values for `fl`,`qt` and `fl_qt`, which is whether I manually determine that whether the sentence is forward-looking, whether it contains time frames and whether it is a forward-looking statement with time frames. The underlying earnings calls' scripts are from the Melbourne Centre for Corporate Governance and Regulation (MCCGR) database. 

In [2]:
from time_frame import is_forward_looking, is_qt_term, print_stats

The following code applies the identification functions to each sentence and shows how `fl_cal`, `qt_cal` and `fl_qt_cal` in `gold_standard` are produced.

In [3]:
gold_standard['fl_cal'] = gold_standard.apply(lambda x: is_forward_looking(x['speaker_text'], x['future_year']), axis=1)
gold_standard['qt_cal'] = gold_standard['speaker_text'].map(is_qt_term)
gold_standard['fl_qt_cal'] = gold_standard.fl_cal & gold_standard.qt_cal

The following function generates statistics to report the performance of the algorithm on identifying forward-looking statements and forward-looking statements with time frames.

In [4]:
training_sample = gold_standard[gold_standard['training sample']==1]
test_sample = gold_standard[gold_standard['training sample']==0]

In [5]:
print_stats(training_sample, type = 'fl')

Accuracy 95.23%
Precision 82.09%
True positive rate 85.94%


In [6]:
print_stats(training_sample, type = 'fl_qt')

Accuracy 96.37%
Precision 88.64%
True positive rate 59.09%


In [7]:
print_stats(test_sample, type = 'fl')

Accuracy 95.82%
Precision 83.61%
True positive rate 87.93%


In [8]:
print_stats(test_sample, type = 'fl_qt')

Accuracy 96.56%
Precision 75.00%
True positive rate 69.23%
